In [1]:
import polars as pl
import os
from pathlib import Path
import numpy as np
import Utils.constants

In [ ]:
import Utils.constants
from Utils.constants import N_LANES, N_SPEEDS, N_TIME_LIMIT, N_DIRECTIONS, N_SPEED_DEVIATION, N_LANE_DEVIATION, N_TIME_LIMIT, FOLDER
from Utils.constants import MAX_COST_VALUE,MASS,K_DRAG, MAX_FORCE
print(f"State space {N_LANES*N_SPEEDS*N_DIRECTIONS} N_SPEED_DEVIATION:{N_SPEED_DEVIATION} N_LANE_DEVIATION:{N_LANE_DEVIATION}")

State space 660 N_SPEED_DEVIATION:2 N_LANE_DEVIATION:1


In [29]:
# Import optimisation data
tc_arr = np.load(Path(FOLDER + f"\\tc_{N_LANES}.npy"))
as_arr= np.load(Path(FOLDER + f"\\as_{N_LANES}.npy"))
ac_arr = np.load(Path(FOLDER + f"\\acc_{N_LANES}.npy"))
_,_,N_TIME = tc_arr.shape

In [30]:
#Bake the parameters into the optimisatoin data
physical_constraints = MASS*ac_arr + K_DRAG*as_arr*as_arr
mask = physical_constraints > MAX_FORCE
tc_arr[mask] = MAX_COST_VALUE
costs_arr = tc_arr.transpose()

In [31]:
#import track data
x_coord_arr = np.full((N_LANES,N_TIME), MAX_COST_VALUE)
y_coord_arr = np.full((N_LANES,N_TIME), MAX_COST_VALUE)

path = Path(FOLDER + f"\\TrackLanes_{N_LANES}.parquet")
track_df = pl.read_parquet(path).limit(N_TIME_LIMIT)
for (l) in range(N_LANES):
    x_coord_arr[l] = track_df[f"x_lane{l}"].to_numpy()
    y_coord_arr[l] = track_df[f"y_lane{l}"].to_numpy()
x_coord_arr = x_coord_arr.transpose()
y_coord_arr = y_coord_arr.transpose()
def get_x_coord(t,l):
    return x_coord_arr[t,l]
def get_y_coord(t,l):
    return y_coord_arr[t,l]

In [32]:
#Helpers
def policy_int_to_tuple(index):
    l = int(index/(N_SPEEDS*N_DIRECTIONS))
    s = int((index-l*(N_SPEEDS*N_DIRECTIONS))/N_DIRECTIONS)
    d = index%N_DIRECTIONS
    return (l,s,d)

def policy_tuple_to_int(l0,s0,d0):
    return l0*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS + d0

def print_state(V):
    for s0 in range(N_SPEEDS):
        for d0 in range(N_DIRECTIONS):
            val = [V[policy_tuple_to_int(l0,s0,d0)] for l0 in range(N_LANES)]
            s = " ".join(f"{x:.5e}" for x in np.ravel(val))
            print(f"s={s0} d={d0}: " + s)

In [33]:
#OPTIMISTION
costs = costs_arr
gamma = 1.0
T, S, A = costs.shape
print(T,S,A)
V = np.zeros((T, S))
policy = np.zeros((T, S), dtype=int)

# terminal time
V[-1] = np.min(costs[-1], axis=1)        # min over actions
policy[-1] = np.argmin(costs[-1], axis=1)
print("Time stamp", T-1)
#print(policy[-1])
#print_state(V[-1])

# backward induction
for t in range(T - 2, -1, -1):
    print("Time stamp", t)
    # q[t, s, a] = C[t, s, a] + gamma * V[t+1, s]
    # We need to broadcast V[t+1] to match actions axis
    cont = gamma * V[t + 1][:, None]     # shape (S, 1)
    q = costs[t] + cont                  # shape (S, A)
    policy[t] = np.argmin(q, axis=1)
    #print(policy[t])
    V[t] = np.min(q, axis=1)
    #print(V[t])
# print(V)
# print(policy)
print(V.shape, policy.shape)
#return V, policy

1178 660 660
Time stamp 1177
Time stamp 1176
Time stamp 1175
Time stamp 1174
Time stamp 1173
Time stamp 1172
Time stamp 1171
Time stamp 1170
Time stamp 1169
Time stamp 1168
Time stamp 1167
Time stamp 1166
Time stamp 1165
Time stamp 1164
Time stamp 1163
Time stamp 1162
Time stamp 1161
Time stamp 1160
Time stamp 1159
Time stamp 1158
Time stamp 1157
Time stamp 1156
Time stamp 1155
Time stamp 1154
Time stamp 1153
Time stamp 1152
Time stamp 1151
Time stamp 1150
Time stamp 1149
Time stamp 1148
Time stamp 1147
Time stamp 1146
Time stamp 1145
Time stamp 1144
Time stamp 1143
Time stamp 1142
Time stamp 1141
Time stamp 1140
Time stamp 1139
Time stamp 1138
Time stamp 1137
Time stamp 1136
Time stamp 1135
Time stamp 1134
Time stamp 1133
Time stamp 1132
Time stamp 1131
Time stamp 1130
Time stamp 1129
Time stamp 1128
Time stamp 1127
Time stamp 1126
Time stamp 1125
Time stamp 1124
Time stamp 1123
Time stamp 1122
Time stamp 1121
Time stamp 1120
Time stamp 1119
Time stamp 1118
Time stamp 1117
Time stamp 

In [35]:
#Can we print the optimal path from the start
l0 = int(N_LANES/2)
s0 = 0
d0 = 2

#Gathers stats about optimal path and speed
racing_stats = np.full((N_TIME,4), 0.0)

for t in range(N_TIME):
    index = policy_tuple_to_int(l0,s0,d0)
    #print(index)
    cost = V[t][index]
    p = policy[t][index]
    # print("state of V")
    #print(V[t])
    #print(policy[t])
    print(f"{t}, time {cost}, lane {l0} speed {s0} direction {d0}, index {index} policy {p}  {policy_int_to_tuple(p)}")
    l0,s0,d0 = policy_int_to_tuple(p)
    #fill out racing stat
    racing_stats[t,0] = l0
    racing_stats[t,1] = get_x_coord(t,l0)
    racing_stats[t,2] = get_y_coord(t,l0)
    racing_stats[t,3] = s0

0, time 22.647596019732426, lane 5 speed 0 direction 2, index 302 policy 361  (6, 0, np.int64(1))
1, time 22.598973565855587, lane 6 speed 0 direction 1, index 361 policy 302  (5, 0, np.int64(2))
2, time 22.608903893610822, lane 5 speed 0 direction 2, index 302 policy 242  (4, 0, np.int64(2))
3, time 22.622006685117004, lane 4 speed 0 direction 2, index 242 policy 301  (5, 0, np.int64(1))
4, time 22.57021130814521, lane 5 speed 0 direction 1, index 301 policy 242  (4, 0, np.int64(2))
5, time 22.583314573457645, lane 4 speed 0 direction 2, index 242 policy 301  (5, 0, np.int64(1))
6, time 22.531518654293116, lane 5 speed 0 direction 1, index 301 policy 242  (4, 0, np.int64(2))
7, time 22.544622871202655, lane 4 speed 0 direction 2, index 242 policy 301  (5, 0, np.int64(1))
8, time 22.492826606911443, lane 5 speed 0 direction 1, index 301 policy 242  (4, 0, np.int64(2))
9, time 22.505931785007185, lane 4 speed 0 direction 2, index 242 policy 301  (5, 0, np.int64(1))
10, time 22.454135371

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

for i in range(N_LANES):
    fig.add_trace(go.Scatter(
        x = track_df["x_lane" + str(i)].to_numpy(),
        y = track_df["y_lane" + str(i)].to_numpy(),
        mode = "markers",
        marker  = dict(size=3, color="lightgray"),
        name = "lane " + str(i)
    ))

x_coord_car = racing_stats.transpose()[1]
y_coord_car = racing_stats.transpose()[2]

fig.add_trace(go.Scatter(
    x=x_coord_car,
    y=y_coord_car,
    mode="lines",
    line=dict(color="red", width=1),
    name=f"car"
))

fig.update_layout(
    width=1200,
    height=1000,
    title="Track Lanes",
    xaxis=dict(scaleanchor="y", scaleratio=1),
)
fig.show()

Garbage below

In [ ]:
verbose = False
def l0_range():
    return range(N_LANES)
def l1_range(index):
    return range(max(0,index-N_LANE_DEVIATION), min(N_LANES,index+N_LANE_DEVIATION+1))
distance_pairs = [(l0,l1) for l0 in l0_range() for l1 in l1_range(l0)]
if verbose and False:
    for (l0,l1) in distance_pairs:
        print(f"{l0} -> {l1}")
    if verbose:
        # print("Distance pairs")
        for (l0,l1) in distance_pairs:
            print(l0,l1)

def d_range(l0):
     return [index for index in range(N_DIRECTIONS) if (direction_to_lane_change(index) + l0 >= 0 and direction_to_lane_change(index) + l0 < N_LANES)]  
if verbose and False:
    print("Distance pairs")
    for l0 in range(N_LANES):
        for d0 in d_range(l0):
             print(l0,l0+direction_to_lane_change(d0))

def s0_range():
    return range(N_SPEEDS)
def s1_range(index):
    return range(max(0,index-N_SPEED_DEVIATION), min(N_SPEEDS,index+N_SPEED_DEVIATION+1))

#state*action space
action_range = [(l0,s0,d0,l0+direction_to_lane_change(d0),s1,d1) for l0 in l0_range() for s0 in s0_range() for d0 in d_range(l0) for s1 in s1_range(s0) for d1 in d_range(l0+direction_to_lane_change(d0)) ]
if verbose:
    print("state by action space", len(action_range))
    for (l0,s0,d0,l1,s1,d1) in action_range:
        print(f"{l0} {s0} {d0} -> {l1} {s1} {d1}")


In [ ]:
#distance between lanes in subsequent time steps
def make_lane_distance_columns(l0,l1):
    x_col0 = pl.col("x_lane"+str(l0))
    x_col1 = pl.col("x_lane"+str(l1)).shift(-1)
    y_col0 = pl.col("y_lane"+str(l0))
    y_col1 = pl.col("y_lane"+str(l1)).shift(-1)
    return ((x_col1-x_col0).pow(2) + (y_col1-y_col0).pow(2)).sqrt().fill_null(strategy="forward").alias(f"d_l{l0}_l{l1}")
lane_distance_columns = [make_lane_distance_columns(l0,d0) for (l0,d0) in distance_pairs]

#distance between lanes in subsequent time steps
def make_velocity_change_column(l0,s0,d0,l1,s1,d1):
    l2 = l1+direction_to_lane_change(d1)
    speed0 = index_to_speed(s0)
    speed1 = index_to_speed(s1)
    x_col0 = pl.col("x_lane"+str(l0))
    x_col1 = pl.col("x_lane"+str(l1)).shift(-1).fill_null(strategy="forward")
    x_col2 = pl.col("x_lane"+str(l2)).shift(-2).fill_null(strategy="forward")
    y_col0 = pl.col("y_lane"+str(l0))
    y_col1 = pl.col("y_lane"+str(l1)).shift(-1).fill_null(strategy="forward")
    y_col2 = pl.col("y_lane"+str(l2)).shift(-2).fill_null(strategy="forward")
    d01_col = pl.col(f"d_l{l0}_l{l1}")
    d12_col = pl.col(f"d_l{l1}_l{l2}")
    return ((speed0*(x_col2-x_col1)/d12_col - speed1*(x_col1-x_col0)/d01_col).pow(2) + 
            (speed0*(y_col2-y_col1)/d12_col - speed1*(y_col1-y_col0)/d01_col).pow(2)).sqrt().fill_null(pl.lit(0)).alias(f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
velocity_change_columns = [make_velocity_change_column(l0,s0,d0,l1,s1,d1) for (l0,s0,d0,l1,s1,d1) in action_range]
#make name functions for these
velocity_change_columns_names = [f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}" for (l0,s0,d0,l1,s1,d1) in action_range]

#average speed
def make_average_speed(s0,s1):
    speed0 = index_to_speed(s0)
    speed1 = index_to_speed(s1)
    return (0.5*(pl.lit(speed0) + pl.lit(speed1))).alias(f"as_s{s0}_s{s1}")
average_speeds_columns = [make_average_speed(s0,s1) for s0 in s0_range() for s1 in s1_range(s0)]

#time cost
def make_time_costs(l0,l1,s0,s1):
    av_speed = pl.col(f"as_s{s0}_s{s1}")
    d01_col = pl.col(f"d_l{l0}_l{l1}")
    return pl.when(d01_col == 0).then(MAX_COST_VALUE).otherwise(av_speed/d01_col).alias(f"tc_l{l0}_{l1}_s{s0}_s{s1}")
time_costs_columns = [make_time_costs(l0,l1,s0,s1) for l0 in l0_range() for l1 in l1_range(l0) for s0 in s0_range() for s1 in s1_range(s0)]

def make_acceleration(l0,s0,d0,l1,s1,d1):
    velocity_change = pl.col(f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
    time_cost = pl.col(f"tc_l{l0}_{l1}_s{s0}_s{s1}")
    return (velocity_change/time_cost).alias(f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
acceleration_columns = [make_acceleration(l0,s0,d0,l1,s1,d1) for (l0,s0,d0,l1,s1,d1) in action_range]

#We need to keep average speeds, time_costs, acceleration for the optimisation


adf = df.lazy().with_columns(lane_distance_columns).with_columns(velocity_change_columns).with_columns(average_speeds_columns).with_columns(time_costs_columns).with_columns(acceleration_columns)
adf1 = adf.collect()
adf1

In [ ]:
path = Path(FOLDER + f"\\adf1_{N_LANES}.parquet")
#adf1.drop(velocity_change_columns_names).write_parquet(path)
adf1.write_parquet(path)

In [ ]:
adf1 = pl.read_parquet(path)

In [ ]:
#arrays with coordinates
x_coord_arr = np.full((N_LANES,N_TIME), MAX_COST_VALUE)
y_coord_arr = np.full((N_LANES,N_TIME), MAX_COST_VALUE)
for (l) in range(N_LANES):
    x_coord_arr[l] = list(adf1[f"x_lane{l}"])
    y_coord_arr[l] = list(adf1[f"y_lane{l}"])
x_coord_arr = x_coord_arr.transpose()
y_coord_arr = y_coord_arr.transpose()
def get_x_coord(t,l):
    return x_coord_arr[t,l]
def get_y_coord(t,l):
    return y_coord_arr[t,l]

In [ ]:
#array with distances
distances_arr = np.full((N_LANES*N_LANES,N_TIME), MAX_COST_VALUE)
for (l0,l1) in distance_pairs:
    index = l0*N_LANES + l1
    distances_arr[index] = list(adf1[f"d_l{l0}_l{l1}"])
distances_arr = distances_arr.transpose()
def get_distance(t,l0,l1):
    index = l0*N_LANES+(l1)
    return distances_arr[t,index]
print(distances_arr.shape)
if verbose:
    for l0 in range(N_LANES):
        print(f"{distances_arr[2,l0*N_LANES+0]} {distances_arr[2,l0*N_LANES+1]} {distances_arr[2,l0*N_LANES+2]}")

In [ ]:
#Tools to navigate the cost array
def make_cost_array(): return np.full((N_LANES*N_SPEEDS*N_DIRECTIONS,N_LANES*N_SPEEDS*N_DIRECTIONS,N_TIME), MAX_COST_VALUE)
#costs_arr = make_cost_array()

tc_arr = make_cost_array()
acc_arr = make_cost_array()
as_arr = make_cost_array()

def state_cost_tuple_to_index(l0,s0,d0):
    index = l0*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS + d0 
    return index

for (l0,s0,d0,l1,s1,d1) in action_range:
    index0 = state_cost_tuple_to_index(l0,s0,d0) 
    index1 = state_cost_tuple_to_index(l1,s1,d1)
    tc_arr[index0][index1] = adf1[f"tc_l{l0}_{l1}_s{s0}_s{s1}"].to_numpy()
    acc_arr[index0][index1] = adf1[f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"].to_numpy()
    as_arr[index0][index1] = adf1[f"as_s{s0}_s{s1}"].to_numpy()

In [ ]:
# #Tools to navigate the cost array
# def make_cost_array(): return np.full((N_LANES*N_SPEEDS*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS,N_TIME), MAX_COST_VALUE)
# costs_arr = make_cost_array()

# def action_cost_tuple_to_index(l0,s0,d0,l1,s1,d1):
#     index = l0*N_SPEEDS*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS + d0*N_LANES*N_SPEEDS*N_DIRECTIONS + l1*N_SPEEDS*N_DIRECTIONS + s1*N_DIRECTIONS + d1 
#     return index

In [ ]:
# #time cost array
# print("time cost array")
# tc_arr = make_cost_array()
# for (l0,s0,d0,l1,s1,d1) in action_range:
#     index = action_cost_tuple_to_index(l0,s0,d0,l1,s1,d1) 
#     tc_arr[index] = adf1[f"tc_l{l0}_{l1}_s{s0}_s{s1}"]
    
# #average speed array
# print("average speet array")
# as_arr = make_cost_array()
# for (l0,s0,d0,l1,s1,d1) in action_range:
#     index = action_cost_tuple_to_index(l0,s0,d0,l1,s1,d1) 
#     as_arr[index] = adf1[f"as_s{s0}_s{s1}"]

# #acceleration array
# print("acceleration array")
# acc_arr = make_cost_array()
# for (l0,s0,d0,l1,s1,d1) in action_range:
#     index = action_cost_tuple_to_index(l0,s0,d0,l1,s1,d1) 
#     acc_arr[index] = adf1[f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"]

In [ ]:
as_arr_2 = as_arr
tc_arr_2 = tc_arr
acc_arr_2 = acc_arr

In [ ]:
np.save(Path(FOLDER + f"\\tc_{N_LANES}"), tc_arr)
np.save(Path(FOLDER + f"\\as_{N_LANES}"), as_arr)
np.save(Path(FOLDER + f"\\acc_{N_LANES}"), acc_arr)

In [7]:
tc_arr = np.load(Path(FOLDER + f"\\tc_{N_LANES}.npy"))
as_arr= np.load(Path(FOLDER + f"\\as_{N_LANES}.npy"))
ac_arr = np.load(Path(FOLDER + f"\\acc_{N_LANES}.npy"))

In [8]:
physical_constraints = MASS*ac_arr + K_DRAG*as_arr*as_arr

In [18]:
physical_constraints[0][2]

array([1278.84583251, 1279.14987011, 1279.40987449, ..., 1278.48069579,
       5159.26777621, 1267.17356529])

In [20]:
mask = physical_constraints > MAX_FORCE
tc_arr[mask] = MAX_COST_VALUE
costs_arr = tc_arr

In [22]:
costs_arr[0][0]

array([0.01999817, 0.01999745, 0.01999797, ..., 0.01999488, 0.01999703,
       0.01999703])

In [ ]:

# #Need a cost array time*states*actions
# velocity_deltas_arr = np.full((N_LANES*N_SPEEDS*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS,N_TIME), MAX_COST_VALUE)
# #def action_index_to_tuple():
# def action_cost_tuple_to_index(l0,s0,d0,l1,s1,d1):
#     index = l0*N_SPEEDS*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS + d0*N_LANES*N_SPEEDS*N_DIRECTIONS + l1*N_SPEEDS*N_DIRECTIONS + s1*N_DIRECTIONS + d1 
#     return index
# for (l0,s0,d0,l1,s1,d1) in action_range:
#     index = action_cost_tuple_to_index(l0,s0,d0,l1,s1,d1) 
#     velocity_deltas_arr[index] = list(adf1[f"C_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"])
# velocity_deltas_arr = velocity_deltas_arr.transpose()
# def get_velocity(t,l0,s0,d0,l1,s1,d1):
#     index = action_cost_tuple_to_index(l0,s0,d0,l1,s1,d1)
#     return velocity_deltas_arr[t,index]
# # for (l0,s0,d0,s1,d1) in action_range:
# #     if l0 == l0+d0 == 1:
# #         index = action_cost_tuple_to_index(l0,s0,d0,s1,d1) 
# #         index2 = l0*N_SPEEDS*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS*N_LANES*N_SPEEDS*N_DIRECTIONS + d0*N_LANES*N_SPEEDS*N_DIRECTIONS + (l0+d0)*N_SPEEDS*N_DIRECTIONS + s1*N_DIRECTIONS + d1 
# #         print(f"{l0} {s0} {d0} -> {l0+d0} {s1} {d1} cost: {velocity_deltas_arr[0,index]} {velocity_deltas_arr[0,index2]}")
        

In [ ]:
# #stupid
# def make_action_list(s):
#     return [MAX_VALUE for x in range(N_LANES*N_SPEEDS*N_DIRECTIONS)]
# costs_list = [[make_action_list(s) for s in range(N_LANES*N_SPEEDS*N_DIRECTIONS)] for t in range(N_TIME)]
# import numpy as np
# costs_arr = np.array(costs_list)
# costs_arr.shape

In [ ]:
# #optimiser quantities
# MASS = 1000
# MAX_FORCE = 80000000
# K_DRAG = 100

# #cost array for Diffie-Bellman (iteration order means it is dog slow. can fix later)
# for t in range(N_TIME):
#     print(t)
#     distances = distances_arr[t]
#     velocity_deltas = velocity_deltas_arr[t]
#     for (l0,s0,d0,l1,s1,d1) in action_range:
#         distance = get_distance(t,l0,l1)
#         velocity_delta = get_velocity(t,l0,s0,d0,l1,s1,d1)
#         speed0 = index_to_speed(s0)
#         speed1 = index_to_speed(s1)
#         average_speed = 0.5*(speed0 + speed1)
#         # if average_speed == 0: 
#         #     print(index_t, index_0, index_1, "average speed zero", average_speed)
#         time_cost = distance / average_speed if distance > 0 else 0.0
        
#         # if time_cost == 0: 
#         #     print(index_t, index_0, index_1, "average speed zero", time_cost)
#         acc = velocity_delta/time_cost
#         force_required = acc*MASS + average_speed*average_speed*K_DRAG
#         force_cost = 0
#         if force_required > MAX_FORCE:
#                 force_cost = MAX_VALUE
#         index1 = l0*N_SPEEDS*N_DIRECTIONS+s0*N_DIRECTIONS+d0,
#         index2 = l1*N_SPEEDS*N_DIRECTIONS + s1*N_DIRECTIONS + d1
#         #print(f"time {t} from {l0} {s0} {d0} -> {l1} {s1} {d1} with d={distance} as={average_speed} c={time_cost} constraint={force_cost}")
#         costs_arr[t,index1,index2] = time_cost + force_cost
# costs_arr[-1]

In [23]:
# num_rows, num_columns = costs_arr[0].shape
# num_rows * num_columns
# costs_arr[-1][4]
def policy_int_to_tuple(index):
    l = int(index/(N_SPEEDS*N_DIRECTIONS))
    s = int((index-l*(N_SPEEDS*N_DIRECTIONS))/N_DIRECTIONS)
    d = index%N_DIRECTIONS
    return (l,s,d)

def policy_tuple_to_int(l0,s0,d0):
    return l0*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS + d0

def print_state(V):
    for s0 in range(N_SPEEDS):
        for d0 in range(N_DIRECTIONS):
            val = [V[policy_tuple_to_int(l0,s0,d0)] for l0 in range(N_LANES)]
            s = " ".join(f"{x:.5e}" for x in np.ravel(val))
            print(f"s={s0} d={d0}: " + s)

In [24]:
costs = costs_arr
gamma = 1.0
T, S, A = costs.shape
V = np.zeros((T, S))
policy = np.zeros((T, S), dtype=int)

# terminal time
V[-1] = np.min(costs[-1], axis=1)        # min over actions
policy[-1] = np.argmin(costs[-1], axis=1)
print("Time stamp", T-1)
print(policy[-1])
print_state(V[-1])

# backward induction
for t in range(T - 2, -1, -1):
    print("Time stamp", t)
    # q[t, s, a] = C[t, s, a] + gamma * V[t+1, s]
    # We need to broadcast V[t+1] to match actions axis
    cont = gamma * V[t + 1][:, None]     # shape (S, 1)
    q = costs[t] + cont                  # shape (S, A)
    policy[t] = np.argmin(q, axis=1)
    print(policy[t])
    V[t] = np.min(q, axis=1)
    #print(V[t])
# print(V)
# print(policy)

#return V, policy

Time stamp 659
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0

In [25]:
print("State 0")
print_state(V[0])

print("State 1")
print_state(V[1])

print("State 2")
print_state(V[2])

# for s0 in range(N_SPEEDS):
#     for d0 in range(N_DIRECTIONS):
#         val = [V[t,policy_tuple_to_int(l0,s0,d0)] for l0 in range(N_LANES)]
#         s = " ".join(f"{x:.5e}" for x in np.ravel(val))
#         print(f"s={s0} d={d0}: " + s)

State 0
s=0 d=0: 6.54000e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.54000e+06
s=0 d=1: 6.60000e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.54000e+06
s=0 d=2: 6.54000e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.51001e+06 6.60000e+06
s=1 d=0: 6.52001e+06 6.48002e+06 6.48002e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48002e+06 6.48002e+06 6.52002e+06
s=1 d=1: 6.60000e+06 6.48002e+06 6.48002e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48002e+06 6.48002e+06 6.52002e+06
s=1 d=2: 6.52001e+06 6.48002e+06 6.48002e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48003e+06 6.48002e+06 6.48002e+06 6.60000e+06
s=2 d=0: 6.50003e+06 6.45004e+06 6.45005e+06 6.45005e+06 6.45005e+06 6.45006e+06 6.45005e+06 6.45005e+06 6.45005e+06 6.45005e+06 6.50003e+06
s=2 d

In [26]:
#Can we print the optimal path from the start
l0 = int(N_LANES/2)
s0 = 0
d0 = 2

#Gathers stats about optimal path and speed
racing_stats = np.full((N_TIME,4), 0.0)

for t in range(N_TIME):
    index = policy_tuple_to_int(l0,s0,d0)
    #print(index)
    cost = V[t][index]
    p = policy[t][index]
    # print("state of V")
    #print(V[t])
    #print(policy[t])
    print(f"{t}, time {cost}, lane {l0} speed {s0} direction {d0}, index {index} policy {p}  {policy_int_to_tuple(p)}")
    l0,s0,d0 = policy_int_to_tuple(p)
    #fill out racing stat
    racing_stats[t,0] = l0
    racing_stats[t,1] = get_x_coord(t,l0)
    racing_stats[t,2] = get_y_coord(t,l0)
    racing_stats[t,3] = s0

0, time 6510008.550405153, lane 5 speed 0 direction 2, index 302 policy 0  (0, 0, np.int64(0))


NameError: name 'get_x_coord' is not defined

In [ ]:
racing_stats.transpose()[1]
racing_stats.transpose()[2]

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

for i in range(N_LANES):
    fig.add_trace(go.Scatter(
        x = df["x_lane" + str(i)].to_numpy(),
        y = df["y_lane" + str(i)].to_numpy(),
        mode = "markers",
        marker  = dict(size=3, color="lightgray"),
        name = "lane " + str(i)
    ))

x_coord_car = racing_stats.transpose()[1]
y_coord_car = racing_stats.transpose()[2]

fig.add_trace(go.Scatter(
    x=x_coord_car,
    y=y_coord_car,
    mode="lines",
    line=dict(color="red", width=1),
    name=f"car"
))

fig.update_layout(
    width=1200,
    height=1000,
    title="Track Lanes",
    xaxis=dict(scaleanchor="y", scaleratio=1),
)

fig.show()

In [ ]:
print(racing_stats[0])
print(racing_stats[1])
print(racing_stats[2])


In [ ]:


for t in range(N_TIME_LIMIT):
    v = V[t]
    print(v)
    m = np.argmin(v)    
    print(m)
    p = policy_format_change(m)
    print(p)    

In [ ]:
def policy_format_change(index):
    l = int(index/(N_SPEEDS*N_DIRECTIONS))
    s = (index-l*(N_SPEEDS*N_DIRECTIONS))/N_DIRECTIONS
    d = index%N_DIRECTIONS
    return (l,s,d)

V[0]

In [ ]:
print(policy_format_change(np.argmin(V[0])))
print(policy_format_change(np.argmin(V[1])))
print(policy_format_change(np.argmin(V[2])))
print(policy_format_change(np.argmin(V[3])))
print(policy_format_change(np.argmin(V[4])))
print(policy_format_change(np.argmin(V[5])))
print(policy_format_change(np.argmin(V[6])))
print(policy_format_change(np.argmin(V[7])))
print(policy_format_change(np.argmin(V[8])))

In [ ]:
action_range_for_time_state = [(l0,s0,d0,s1,d1) for l0 in l1_range() for s0 in s1_range() for d0 in d_range(l0) for s1 in s2_range(s0) for d1 in d_range(l0+d0) ]

def cost_of_action(t,l0,s0,d0,s1,d1):
    distance = distances_arr[index_t,lane0*N_LANES + lane1]
    average_speed = 0.5*(index_to_speed(speed0) + index_to_speed(speed1))
    if average_speed == 0: 
        print(index_t, index_0, index_1, "average speed zero", average_speed)
    time_cost = distance / average_speed if distance > 0 else 0.0
    if time_cost == 0: 
        print(index_t, index_0, index_1, "average speed zero", time_cost)
    acc = (index_to_speed(speed1) - index_to_speed(speed0))/time_cost
    force_required = acc*MASS + average_speed*average_speed*K_DRAG
    force_cost = 0
    if force_required > MAX_FORCE:
        force_cost = 1000.0
    costs[index_t, index_0, index_1] = time_cost + force_cost
    
def compute_cost_of_action(t,(l0,s0,d0)):
    actions = [ (s1,d2) for s1 in s2_range(s0) for d1 in d_range(l0+d0)  ]

    


In [ ]:
#Need the cost array
max_value = 1000000
action_costs = []
for t in range(N_TIME):
    states = [max_value for index in range(N_LANES*N_SPEEDS*N_DIRECTIONS)]
    action_costs.append(states)

    for l in range(N_LANES):
        for s in range(N_SPEEDS):
            for d in range(N_SPEEDS):

    action_costs.append( compute_cost_for_action(a) )  # each is (T, S)

# action_costs = []
# for a in actions:
#     action_costs.append( compute_cost_for_action(a) )  # each is (T, S)

# stack them once you know all of them:

# C = np.stack(action_costs, axis=-1) 

In [ ]:
costs = np.zeros((N_TIME, N_LANES*N_SPEEDS, N_LANES*N_SPEEDS), dtype=np.float32)
for index_t in range(N_TIME):
    for index_0 in range(N_LANES*N_SPEEDS):
        lane0,speed0 = index_lane_speed(index_0)
        for index_1 in range(N_LANES*N_SPEEDS):
            lane1,speed1 = index_lane_speed(index_1)
            distance = distances_arr[index_t,lane0*N_LANES + lane1]
            average_speed = 0.5*(index_to_speed(speed0) + index_to_speed(speed1))
            if average_speed == 0: 
                print(index_t, index_0, index_1, "average speed zero", average_speed)
            time_cost = distance / average_speed if distance > 0 else 0.0
            if time_cost == 0: 
                print(index_t, index_0, index_1, "average speed zero", time_cost)
            acc = (index_to_speed(speed1) - index_to_speed(speed0))/time_cost
            force_required = acc*MASS + average_speed*average_speed*K_DRAG
            force_cost = 0
            if force_required > MAX_FORCE:
                force_cost = 1000.0
            costs[index_t, index_0, index_1] = time_cost + force_cost

In [ ]:
[(i0,i1,i2,i3,i4,i5) for i0 in range(N_DIRECTIONS) for i1 in range(N_DIRECTIONS) for i2 in range(N_LANES) for i3 in range(max(0,i2-N_LANE_DEVIATION), min(N_LANES-1,i2+N_LANE_DEVIATION)) for i4 in range(N_SPEEDS) for i5 in range(max(0,i4-N_SPEED_DEVIATION), min(N_SPEEDS-1,i4+N_SPEED_DEVIATION)) ]
        

In [ ]:
distance_columns = [f"distance_{i0}_{i1}" for i0 in range(N_LANES) for i1 in range(N_LANES)]
distances_arr = adf1.select(distance_columns).to_numpy()
distances_arr

In [ ]:
# For the dyanmic programming, we need a matrix with the transition costs for each time step
N_TIME = len(adf.collect())
import numpy as np
def index_lane_speed(index):
    lane = int(index/N_SPEEDS)
    speed = index-lane*N_SPEEDS
    return int(lane), int(speed)

costs = np.zeros((N_TIME, N_LANES*N_SPEEDS, N_LANES*N_SPEEDS), dtype=np.float32)
for index_t in range(N_TIME):
    for index_0 in range(N_LANES*N_SPEEDS):
        lane0,speed0 = index_lane_speed(index_0)
        for index_1 in range(N_LANES*N_SPEEDS):
            lane1,speed1 = index_lane_speed(index_1)
            distance = distances_arr[index_t,lane0*N_LANES + lane1]
            average_speed = 0.5*(index_to_speed(speed0) + index_to_speed(speed1))
            if average_speed == 0: 
                print(index_t, index_0, index_1, "average speed zero", average_speed)
            time_cost = distance / average_speed if distance > 0 else 0.0
            if time_cost == 0: 
                print(index_t, index_0, index_1, "average speed zero", time_cost)
            acc = (index_to_speed(speed1) - index_to_speed(speed0))/time_cost
            force_required = acc*MASS + average_speed*average_speed*K_DRAG
            force_cost = 0
            if force_required > MAX_FORCE:
                force_cost = 1000.0
            costs[index_t, index_0, index_1] = time_cost + force_cost


In [ ]:
len(distances_arr[0])

In [ ]:


df = aldf.collect()
columns = dict([(i,name) for name,i in enumerate(df.columns)])
print(columns)
arr = df.to_numpy()
arr.shape

In [ ]:
df = tldf.collect()
columns = dict([(i,name) for name,i in enumerate(df.columns)])
arr = tldf.collect().to_numpy()
arr

T, S, A = costs.shape
V = np.zeros((T, S))
policy = np.zeros((T, S), dtype=int)

# terminal time
V[-1] = np.min(costs[-1], axis=1)        # min over actions
policy[-1] = np.argmin(costs[-1], axis=1)

# backward induction
for t in range(T - 2, -1, -1):
    # q[t, s, a] = C[t, s, a] + gamma * V[t+1, s]
    # We need to broadcast V[t+1] to match actions axis
    cont = gamma * V[t + 1][:, None]     # shape (S, 1)
    q = costs[t] + cont                  # shape (S, A)

    policy[t] = np.argmin(q, axis=1)
    V[t] = np.min(q, axis=1)

return V, policy

In [ ]:
import numpy as np

T, S = arr.shape
print(T,S)
A = N_LANES*N_SPEEDS
print(A)

def cost_for_action(X, a):
    print(X.shape)
    if a == 0:
        return X
    elif a == 1:
        return 2 * X + 1
    elif a == 2:
        return X**2
    else:
        raise ValueError("Unknown action")

C = np.empty((T, S, A))
action_d = {}
for i in range(N_LANES): 
    for j in range(N_SPEEDS): 
        a = i*N_LANES + j
        action_d["{i}_{j}"] = i*N_LANES + j
        C[..., a] = cost_for_action(arr, a)
        
# for a in range(A):
#     C[..., a] = cost_for_action(arr, a)
arr

In [ ]:
#how much can we add in terms of transition data
N_SPEEDS = 21
tldf = aldf
print(tldf.collect().estimated_size("mb"))
tldf = tldf.with_columns(
        [(pl.col("x_m")).alias(f"lane_{i}_speed_{j}") for i,f in enumerate(lane_factors) for j in range(N_SPEEDS)]
)
print(tldf.collect().estimated_size("mb"))


In [ ]:
df = tldf.collect()
columns = dict([(i,name) for name,i in enumerate(df.columns)])
arr = tldf.collect().to_numpy().transpose()
arr
N = len(arr[0])
def exp(i,j):
    neighbours = [
        (j0, j1, pl.col(f"distance_{i0}_{j0}") +  pl.col(f"min_cost_l{i0}s{j0}").shift(-1))
        for j0 in range(max(0, i0 - 2), min(N_LANES, i0 + 3))
        for j1 in range(max(0, i1 - 2), min(N_SPEEDS, i1 + 3))
    ]

    dist_exprs = [d for (_, _, d) in neighbours]
    min_dist = pl.min_horizontal(dist_exprs)

    # Build both j0 and j1 CASE expressions in the same loop
    
    j0_case = None
    j1_case = None
    for j0, j1, dist in neighbours:
        cond = (dist == min_dist)

        if j0_case is None:
            j0_case = pl.when(cond).then(j0)
        else:
            j0_case = j0_case.when(cond).then(j0)

        if j1_case is None:
            j1_case = pl.when(cond).then(j1)
        else:
            j1_case = j1_case.when(cond).then(j1)
    
    # Finalize both with .otherwise(None)
    j0_case = j0_case.otherwise(None)
    j1_case = j1_case.otherwise(None)
            
    return [
        min_dist.alias(f"min_cost_l{i0}s{i1}"),
        j0_case.alias(f"min_cost_l{i0}s{i1}_j0"),
        j1_case.alias(f"min_cost_l{i0}s{i1}_j
                      

    return 0
d = {}
for i in range(N_LANES): 
    for j in range(N_SPEEDS): 
        l = [exp(i) for i in range(N)]
        # "min_cost_l{i0}s{i1}"
        # "min_cost_l{i0}s{i1}_j0"
        # "min_cost_l{i0}s{i1}_j1"
        #tldf2 = tldf2.with_columns( cost_expr(i,j) )
        d["min_cost_l{i0}s{i1}"] = l
d


In [ ]:

def cost_expr(i0: int, i1: int) -> list[pl.Expr]:
    # neighbourhood list
    neighbours = [
        (j0, j1, pl.col(f"distance_{i0}_{j0}") +  pl.col(f"min_cost_l{i0}s{j0}").shift(-1))
        for j0 in range(max(0, i0 - 2), min(N_LANES, i0 + 3))
        for j1 in range(max(0, i1 - 2), min(N_SPEEDS, i1 + 3))
    ]

    dist_exprs = [d for (_, _, d) in neighbours]
    min_dist = pl.min_horizontal(dist_exprs)

    # Build both j0 and j1 CASE expressions in the same loop
    
    j0_case = None
    j1_case = None
    for j0, j1, dist in neighbours:
        cond = (dist == min_dist)

        if j0_case is None:
            j0_case = pl.when(cond).then(j0)
        else:
            j0_case = j0_case.when(cond).then(j0)

        if j1_case is None:
            j1_case = pl.when(cond).then(j1)
        else:
            j1_case = j1_case.when(cond).then(j1)
    
    # Finalize both with .otherwise(None)
    j0_case = j0_case.otherwise(None)
    j1_case = j1_case.otherwise(None)
            
    return [
        min_dist.alias(f"min_cost_l{i0}s{i1}"),
        j0_case.alias(f"min_cost_l{i0}s{i1}_j0"),
        j1_case.alias(f"min_cost_l{i0}s{i1}_j1"),
    ]
def build_columns():
    columns = []
    for i in range(1): 
        for j in range(1):
            columns.append(cost_expr(i,j))
    return columns

def build_columns(i,j):
    c = cost_expr(i,j)
    #c.append(cost_expr(0,0))
    return c

tldf2 = tldf
for i in range(1): 
    for j in range(1): 
        tldf2 = tldf2.with_columns( cost_expr(i,j) )

tldf2.collect()

In [ ]:
tldf = aldf
print(tldf.collect().estimated_size("mb"))
tldf = tldf.with_columns(
        [((pl.col("x_lane"+str(i1)).shift(-1)-pl.col("x_lane"+str(i0))).pow(2) + 
          (pl.col("y_lane"+str(i1)).shift(-1)-pl.col("y_lane"+str(i0))).pow(2)).sqrt().fill_null(strategy="forward").alias(f"distance_{i0}_{i1}")
         for i0 in range(N_LANES) for i1 in range(N_LANES)]
        )
tldf.collect()

In [ ]:
#Create and add the lanes
ldf = df.lazy()
aldf = ldf.with_columns([
    (pl.col("x_m").shift(-1) - pl.col("x_m").shift(1)).fill_null(strategy="forward").fill_null(strategy="backward").alias("x_dir"),
    (pl.col("y_m").shift(-1) - pl.col("y_m").shift(1)).fill_null(strategy="forward").fill_null(strategy="backward").alias("y_dir"),
    (pl.col("w_tr_left_m") + pl.col("w_tr_right_m")).alias("width")
]).with_columns([
    (pl.col("x_dir")*pl.col("x_dir") + pl.col("y_dir")*pl.col("y_dir")).sqrt().alias("norm_dir")
]).with_columns([
    (pl.col("x_dir")/pl.col("norm_dir")).alias("x_udir"),
    (pl.col("y_dir")/pl.col("norm_dir")).alias("y_udir")
])

N_LANES = 11
lane_factors = [-0.5 + i / (N_LANES - 1) for i in range(N_LANES)]
print(lane_factors)
aldf = aldf.with_columns(
        [(pl.col("x_m") + pl.lit(f) * pl.col("width") * pl.col("y_udir")).alias(f"x_lane{i}") for i, f in enumerate(lane_factors)] +
        [(pl.col("y_m") - pl.lit(f) * pl.col("width") * pl.col("x_udir")).alias(f"y_lane{i}") for i, f in enumerate(lane_factors)]
    )

lanesdfd = aldf.collect()
lanesdfd.describe()
lanesdfd

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

for i in range(N_LANES):
    fig.add_trace(go.Scatter(
        x = lanesdfd["x_lane" + str(i)].to_numpy(),
        y = lanesdfd["y_lane" + str(i)].to_numpy(),
        mode = "markers",
        marker  = dict(size=3),
        name = "lane " + str(i)
    ))

fig.update_layout(
    width=1200,
    height=1000,
    title="Track Lanes",
    xaxis=dict(scaleanchor="y", scaleratio=1),
)

fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(25,25))

plt.plot(lanesdfd["x_lane0"].to_numpy(),
         lanesdfd["y_lane0"].to_numpy(),
         linewidth=0.3,
         label="lane 0")

plt.plot(lanesdfd["x_lane5"].to_numpy(),
         lanesdfd["y_lane5"].to_numpy(),
         linewidth=0.3,
         label="lane 5")

plt.plot(lanesdfd["x_lane10"].to_numpy(),
         lanesdfd["y_lane10"].to_numpy(),
         linewidth=0.3,
         label="lane 10")

plt.legend()
plt.show()

In [ ]:
path = Path(folder_path + r"\\TrackLanes.parquet")
df.write_parquet(path)

In [ ]:
#how much can we add in terms of transition data
N_SPEEDS = 21
tldf = aldf
print(tldf.collect().estimated_size("mb"))
tldf = tldf.with_columns(
        [(pl.col("x_m")).alias(f"lane_{i}_speed_{j}") for i,f in enumerate(lane_factors) for j in range(N_SPEEDS)]
)
print(tldf.collect().estimated_size("mb"))

tldf = aldf
print(tldf.collect().estimated_size("mb"))
tldf = tldf.with_columns(
        [((pl.col("x_lane"+str(i1)).shift(-1)-pl.col("x_lane"+str(i0))).pow(2) + 
          (pl.col("y_lane"+str(i1)).shift(-1)-pl.col("y_lane"+str(i0))).pow(2)).sqrt().fill_null(strategy="forward").alias(f"distance_{i0}_{i1}")
         for i0 in range(N_LANES) for i1 in range(N_LANES)]
        )
tldf.collect()


In [ ]:
tldf = aldf
print(tldf.collect().estimated_size("mb"))
tldf = tldf.with_columns(
        [((pl.col("x_lane"+str(i1)).shift(-1)-pl.col("x_lane"+str(i0))).pow(2) + 
          (pl.col("y_lane"+str(i1)).shift(-1)-pl.col("y_lane"+str(i0))).pow(2)).sqrt().fill_null(strategy="forward").alias(f"distance_{i0}_{i1}")
         for i0 in range(N_LANES) for i1 in range(N_LANES)]
        )
tldf.collect()

In [ ]:

def cost_expr(i0: int, i1: int) -> list[pl.Expr]:
    # neighbourhood list
    neighbours = [
        (j0, j1, pl.col(f"distance_{i0}_{j0}") +  pl.col(f"min_cost_l{i0}s{j0}").shift(-1))
        for j0 in range(max(0, i0 - 2), min(N_LANES, i0 + 3))
        for j1 in range(max(0, i1 - 2), min(N_SPEEDS, i1 + 3))
    ]

    dist_exprs = [d for (_, _, d) in neighbours]
    min_dist = pl.min_horizontal(dist_exprs)

    # Build both j0 and j1 CASE expressions in the same loop
    
    j0_case = None
    j1_case = None
    for j0, j1, dist in neighbours:
        cond = (dist == min_dist)

        if j0_case is None:
            j0_case = pl.when(cond).then(j0)
        else:
            j0_case = j0_case.when(cond).then(j0)

        if j1_case is None:
            j1_case = pl.when(cond).then(j1)
        else:
            j1_case = j1_case.when(cond).then(j1)
    
    # Finalize both with .otherwise(None)
    j0_case = j0_case.otherwise(None)
    j1_case = j1_case.otherwise(None)
            
    return [
        min_dist.alias(f"min_cost_l{i0}s{i1}"),
        j0_case.alias(f"min_cost_l{i0}s{i1}_j0"),
        j1_case.alias(f"min_cost_l{i0}s{i1}_j1"),
    ]
def build_columns():
    columns = []
    for i in range(1): 
        for j in range(1):
            columns.append(cost_expr(i,j))
    return columns

def build_columns(i,j):
    c = cost_expr(i,j)
    #c.append(cost_expr(0,0))
    return c

tldf2 = tldf
for i in range(1): 
    for j in range(1): 
        tldf2 = tldf2.with_columns( cost_expr(i,j) )

tldf2.collect()

In [ ]:
df = tldf.collect()
columns = dict([(i,name) for name,i in enumerate(df.columns)])
arr = tldf.collect().to_numpy().transpose()
arr
N = len(arr[0])
def exp(i,j):
    neighbours = [
        (j0, j1, pl.col(f"distance_{i0}_{j0}") +  pl.col(f"min_cost_l{i0}s{j0}").shift(-1))
        for j0 in range(max(0, i0 - 2), min(N_LANES, i0 + 3))
        for j1 in range(max(0, i1 - 2), min(N_SPEEDS, i1 + 3))
    ]

    dist_exprs = [d for (_, _, d) in neighbours]
    min_dist = pl.min_horizontal(dist_exprs)

    # Build both j0 and j1 CASE expressions in the same loop
    
    j0_case = None
    j1_case = None
    for j0, j1, dist in neighbours:
        cond = (dist == min_dist)

        if j0_case is None:
            j0_case = pl.when(cond).then(j0)
        else:
            j0_case = j0_case.when(cond).then(j0)

        if j1_case is None:
            j1_case = pl.when(cond).then(j1)
        else:
            j1_case = j1_case.when(cond).then(j1)
    
    # Finalize both with .otherwise(None)
    j0_case = j0_case.otherwise(None)
    j1_case = j1_case.otherwise(None)
            
    return [
        min_dist.alias(f"min_cost_l{i0}s{i1}"),
        j0_case.alias(f"min_cost_l{i0}s{i1}_j0"),
        j1_case.alias(f"min_cost_l{i0}s{i1}_j
                      

    return 0
d = {}
for i in range(N_LANES): 
    for j in range(N_SPEEDS): 
        l = [exp(i) for i in range(N)]
        # "min_cost_l{i0}s{i1}"
        # "min_cost_l{i0}s{i1}_j0"
        # "min_cost_l{i0}s{i1}_j1"
        #tldf2 = tldf2.with_columns( cost_expr(i,j) )
        d["min_cost_l{i0}s{i1}"] = l
d


In [ ]:
df = aldf.collect()
columns = dict([(i,name) for name,i in enumerate(df.columns)])
print(columns)
arr = df.to_numpy()
arr.shape

In [ ]:
import numpy as np

T, S = arr.shape
print(T,S)
A = N_LANES*N_SPEEDS
print(A)

def cost_for_action(X, a):
    print(X.shape)
    if a == 0:
        return X
    elif a == 1:
        return 2 * X + 1
    elif a == 2:
        return X**2
    else:
        raise ValueError("Unknown action")

C = np.empty((T, S, A))
action_d = {}
for i in range(N_LANES): 
    for j in range(N_SPEEDS): 
        a = i*N_LANES + j
        action_d["{i}_{j}"] = i*N_LANES + j
        C[..., a] = cost_for_action(arr, a)
        
# for a in range(A):
#     C[..., a] = cost_for_action(arr, a)
arr

In [ ]:
df = tldf.collect()
columns = dict([(i,name) for name,i in enumerate(df.columns)])
arr = tldf.collect().to_numpy()
arr

T, S, A = costs.shape
V = np.zeros((T, S))
policy = np.zeros((T, S), dtype=int)

# terminal time
V[-1] = np.min(costs[-1], axis=1)        # min over actions
policy[-1] = np.argmin(costs[-1], axis=1)

# backward induction
for t in range(T - 2, -1, -1):
    # q[t, s, a] = C[t, s, a] + gamma * V[t+1, s]
    # We need to broadcast V[t+1] to match actions axis
    cont = gamma * V[t + 1][:, None]     # shape (S, 1)
    q = costs[t] + cont                  # shape (S, A)

    policy[t] = np.argmin(q, axis=1)
    V[t] = np.min(q, axis=1)

return V, policy

In [ ]:

def cost_expr(i0: int, i1: int) -> pl.Expr:
    # start with some base
    expr = pl.lit(0)

    expr = expr + pl.min([pl("distance_{i0}_{i1}"), pl("distance_{i0}_{i1}")])
    # add pieces step by step
    expr = expr + pl.lit(10 * i + 1)

    # example: add something depending on speed bin j
    expr = expr + pl.when(pl.col("speed_bin") == j).then(5).otherwise(0)

    # another example: use lane_factors[i]
    # expr = expr + pl.lit(lane_factors[i]) * pl.col("some_column")

    return expr

tldf.with_columns(
        [cost_expr(i,j).alias(f"cost_lane_{i}_speed_{j}")
          for i,f in enumerate(lane_factors) for j in range(20)]
)

In [ ]:
#Adding the transition data
#Create and add the lanes


aldf.collect().estimate_size()

In [ ]:

pl.read_excel("C:\Users\johan\OneDrive\Documents\Python\Python\SilverStone\TrackData.csv")

In [ ]:
from pathlib import Path

my_file = Path(path)
my_file.is_file()